In [39]:
import numpy as np
import math

f_name="circuit.txt"
number_of_wires=8
max_num_controls=0

def get_circuit_from_file(f_name):
    global max_num_controls
    f = open(f_name, "r")
    all_lines=f.readlines()
    arr=[]
    k=0
    for line in all_lines: 
        line=line.rstrip()
        row=[]
        number_of_controls=int(line[0])
        if number_of_controls>max_num_controls:
            max_num_controls=number_of_controls
        target_qubit=int(line[-1])
        if number_of_controls == 0: # not gate
            type_gate=4
            row=[type_gate,[], target_qubit]
        else: # cnot gate with n>1 controls
            type_gate=1
            controls_qubits_as_str =line[2:-2]
            if len(controls_qubits_as_str)==1:
                control_qubit=int(controls_qubits_as_str)
                row=[type_gate,[control_qubit],target_qubit]
            else:
                count=0
                controls_qubit_list=[]
                row.append(type_gate)
                for i in range(0,len(controls_qubits_as_str),2):
                    controls_qubit_list.append(int(controls_qubits_as_str[i]))
                row.append(controls_qubit_list)
                row.append(target_qubit)
        arr.insert(k,row)
        k=k+1
    f.close()
    return arr

def decomposition_ciruit_into_toffoli_gates(all_gates):
    global number_of_wires
    if max_num_controls==number_of_wires-1:
        number_of_wires=number_of_wires+1
    array_levels_after_decomposition=[]
    for gate in all_gates:
        type_gate=gate[0]
        controls=len(gate[1])
        target_q=gate[2]
        if type_gate==1: #cnot gate with n controls
            if controls>2 and number_of_wires>=5:
             con_tar_list=gate[1].copy()
             con_tar_list.append(target_q)
             array_empty_wires=calculate_number_empty_wires(con_tar_list)
             gates=decompose_gate_into_ccx(con_tar_list,array_empty_wires)
             array_levels_after_decomposition=array_levels_after_decomposition+gates
            elif controls==2:
                 con=gate[1]
                 c1=con[0]
                 c2=con[1]
                 g=[type_gate ,c1,c2,target_q]
                 array_levels_after_decomposition.insert(len(array_levels_after_decomposition),g)
            elif controls==1:
                 g=[type_gate ,gate[1],target_q]
                 array_levels_after_decomposition.insert(len(array_levels_after_decomposition),g)                
        elif type_gate==10:
            if controls > 2 and number_of_wires >=5:
                con_tar_list=gate[1].copy()
                con_tar_list.append(target_q)
                array_empty_wires=calculate_number_empty_wires(con_tar_list)
                gates=decompose_inverse_gate_into_ccx(con_tar_list,array_empty_wires)
                array_levels_after_decomposition=array_levels_after_decomposition+gates
            elif controls==2:
                 con=gate[1]
                 c1=con[0]
                 c2=con[1]
                 g=[type_gate ,c1,c2,target_q]
                 array_levels_after_decomposition.insert(len(array_levels_after_decomposition),g)  
            elif controls==1:
                 g=[type_gate ,gate[1],target_q]
                 array_levels_after_decomposition.insert(len(array_levels_after_decomposition),g)      
        else:
            array_levels_after_decomposition.insert(len(array_levels_after_decomposition),gate)
    return array_levels_after_decomposition
            
def calculate_number_empty_wires(arr):
    all_wires=np.arange (number_of_wires)
    empty_wires=np.delete(all_wires,arr)
    return empty_wires

def decompose_inverse_gate_into_ccx(c_t,empty_w):
    gate_in_form_ccx=[]
    n=number_of_wires
    m=len(c_t)-1
    if m>=3 and m<=math.ceil(n/2):
        gate_in_form_ccx=apply_rule_1_inverse(c_t,empty_w)
    elif m==n-2: 
        gate_in_form_ccx_1=apply_rule_2_inverse(c_t,empty_w)
        gate_in_form_ccx=decomposition_ciruit_into_toffoli_gates(gate_in_form_ccx_1)
    else:
        gate_in_form_ccx_1=apply_rule_3(c_t,empty_w)
        gate_in_form_ccx=decomposition_ciruit_into_toffoli_gates(gate_in_form_ccx_1)
    return gate_in_form_ccx

def decompose_gate_into_ccx(c_t,empty_w):
    n=number_of_wires
    m=len(c_t)-1
    if m>=3 and m<=math.ceil(n/2):
        gate_in_form_ccx=apply_rule_1(c_t,empty_w)
    elif m==n-2: 
        gate_in_form_ccx_1=apply_rule_2(c_t,empty_w)
        gate_in_form_ccx=decomposition_ciruit_into_toffoli_gates(gate_in_form_ccx_1)
    else:
        gate_in_form_ccx_1=apply_rule_3(c_t,empty_w)
        gate_in_form_ccx=decomposition_ciruit_into_toffoli_gates(gate_in_form_ccx_1)
    return gate_in_form_ccx

def apply_rule_3(c_t,empty_w): 
    m=len(c_t)
    m1=math.ceil(m/2)
    con_gate1=c_t[0:m1]
    t_gate1=empty_w[-1]
    con_gate2=c_t[m1:-1].copy()
    con_gate2.append(t_gate1)
    t_gate2=c_t[-1]
    gate_in_form_ccx=[]
    gate_in_form_ccx.insert(0,[1, con_gate1, t_gate1])
    gate_in_form_ccx.insert(1,[1, con_gate2, t_gate2])
    gate_in_form_ccx.insert(2,[1, con_gate1, t_gate1])
    gate_in_form_ccx.insert(3,[1, con_gate2, t_gate2])
    return gate_in_form_ccx

def apply_rule_2_inverse(c_t,empty_w):
    m1=math.ceil(number_of_wires/2)
    con_gate1=c_t[0:m1]
    t_gate1=empty_w[-1]
    con_gate2=c_t[m1:-1].copy()
    con_gate2.append(t_gate1)
    t_gate2=c_t[-1]
    gate_in_form_ccx=[]
    gate_in_form_ccx.insert(0,[10, con_gate1, t_gate1])
    gate_in_form_ccx.insert(1,[10, con_gate2, t_gate2])
    gate_in_form_ccx.insert(2,[1 ,con_gate1 ,t_gate1])
    gate_in_form_ccx.insert(3,[1, con_gate2, t_gate2])
    return gate_in_form_ccx

def apply_rule_2(c_t,empty_w):
    m1=math.ceil(number_of_wires/2)
    m2=number_of_wires-m1-1
    con_gate1=c_t[0:m1]
    t_gate1=empty_w[-1]
    con_gate2=c_t[m1:-1].copy()
    con_gate2.append(t_gate1)
    t_gate2=c_t[-1]
    gate_as_ccx=[]
    gate_as_ccx.insert(0,[1, con_gate1, t_gate1])
    gate_as_ccx.insert(1,[1, con_gate2, t_gate2])
    gate_as_ccx.insert(2,[10, con_gate1, t_gate1])
    gate_as_ccx.insert(3,[10, con_gate2, t_gate2])
    return gate_as_ccx

def apply_rule_1(c_t,empty_w):
    c_number=len(c_t)-1
    gate_as_ccx=[]
    if c_number==3:
        gate_as_ccx=[[1, c_t[2],empty_w[0], c_t[3]],[1, c_t[0], c_t[1], empty_w[0]],
                     [10, c_t[2], empty_w[0], c_t[3]],[10, c_t[0], c_t[1], empty_w[0]]]
    elif c_number>3:
        gate_as_ccx.insert(0,[1, c_t[-2], empty_w[c_number-3] , c_t[-1]])
        gate_as_ccx.insert(c_number-2,[1, c_t[0], c_t[1], empty_w[0]])
        i=c_number-3
        k=-1
        for j in range(1,c_number-2):
            c1=c_t[j+1]
            c2=empty_w[j-1]
            t=empty_w[j]
            gate_as_ccx.insert(i,[1,c1,c2,t])
            i=i-1
            gate_as_ccx.insert(c_number+k,[10,c1,c2,t])
            k=k+1
        x=(c_number-2)*2
        gate_as_ccx.insert(x,[10, c_t[-2], empty_w[c_number-3] , c_t[-1]])
        for i in range(1,len(gate_as_ccx)-1):
            gate_as_ccx.append(gate_as_ccx[i])
    return gate_as_ccx

def apply_rule_1_inverse(c_t,empty_w):
    c_number=len(c_t)-1
    gate_as_ccx=[]
    if c_number==3:
        gate_as_ccx=[[ 10, c_t[2], empty_w[0], c_t[-1]],[ 10, c_t[0], c_t[1], empty_w[0]],
                          [ 1, c_t[2], empty_w[0], c_t[-1]],[ 1, c_t[0], c_t[1], empty_w[0]]]
    elif c_number>3:
        gate_as_ccx.insert(0,[10, c_t[-2], empty_w[c_number-3],c_t[-1]])
        gate_as_ccx.insert(c_number-2,[1, c_t[0], c_t[1], empty_w[0]])
        i=c_number-3
        k=-1
        for j in range(1,c_number-2):
            c1=c_t[j+1]
            c2=empty_w[j-1]
            t=empty_w[j]
            gate_as_ccx.insert(i,[10,c1,c2,t])
            i=i-1
            gate_as_ccx.insert(c_number+k,[1,c1,c2,t])
            k=k+1
        x=(c_number-2)*2
        gate_as_ccx.insert(x,[1, c_t[-2], empty_w[c_number-3],c_t[-1]])
        for i in range(1,len(gate_as_ccx)-1):
            gate_as_ccx.append(gate_as_ccx[i])
    return gate_as_ccx

def decomposition_into_elementry_gates(arr_toffoli):
    arr_elementry_gate=[]
    k=0
    for gate in arr_toffoli:
        gate_len=len(gate)
        if gate_len==4:
            gate_type=gate[0]
            con=gate[1:-1]
            target=gate[-1]
            if gate_type==1:
                all_gates=get_elementry_gates(con,target)
                arr_elementry_gate=arr_elementry_gate+all_gates
                k=len(arr_elementry_gate)
            elif gate_type==10:
                all_gates=get_inverse_elementry_gates(con,target)
                arr_elementry_gate=arr_elementry_gate+all_gates
                k=len(arr_elementry_gate)
        else:
           arr_elementry_gate.insert(k,gate)
           k=k+1
    return arr_elementry_gate

def get_inverse_elementry_gates(controls,target):
    elementry_gates=[]
    elementry_gates.insert(0,[3, controls[0], target])
    elementry_gates.insert(1,[1, controls[0], controls[1]])
    elementry_gates.insert(2,[2, controls[1], target])
    elementry_gates.insert(3,[1, controls[0], controls[1]])
    elementry_gates.insert(4,[3, controls[1], target])
    return elementry_gates

def get_elementry_gates(controls,target):
    elementry_gates=[]
    elementry_gates.insert(0,[2, controls[1] ,target])
    elementry_gates.insert(1,[1, controls[0], controls[1]])
    elementry_gates.insert(2,[3, controls[1], target])
    elementry_gates.insert(3,[1, controls[0], controls[1]])
    elementry_gates.insert(4,[2, controls[0], target])
    return elementry_gates

def optimization(arr):
    r=len(arr)
    i=0
    while i<r:
        gate=arr[i]
        gate_type=gate[0]        
        if gate_type==1 :
            cases,check_swap,index_gate2=check_swap_cx(i,gate,arr)
            if cases==0:
                indeces_two_gates=[i, index_gate2]
                arr.pop(indeces_two_gates[0])
                arr.pop(indeces_two_gates[1]-1)
                i=0
            elif cases==2:
                arr[index_gate2]=[ 2, gate[1],gate[2]]
                arr.pop(i)
                i=0
            elif cases==3:
                arr[index_gate2]=[ 3, gate[1],gate[2]]
                arr.pop(i)
                i=0
            elif cases==-1:
                i=i+1
        elif gate_type==2: #cv
            cases,check_swap,index_gate2=check_swap_cv(i,gate,arr)
            if cases==0:
                indeces_two_gates=[i, index_gate2]
                arr.pop(indeces_two_gates[0])
                arr.pop(indeces_two_gates[1]-1)
                i=0
            elif cases==1:
                arr[index_gate2]=[ 1, gate[1],gate[2]]
                arr.pop(i)
                i=0
            elif cases==3:
                arr[index_gate2]=[ 3, gate[1],gate[2]]
                arr.pop(i)
                i=0
            elif cases==-1:
                i=i+1
        elif gate_type==3: #cvp
            cases,check_swap,index_gate2=check_swap_cvp(i,gate,arr)
            if cases==0:
               indeces_two_gates=[i, index_gate2]
               arr.pop(indeces_two_gates[0])
               arr.pop(indeces_two_gates[1]-1)
               i=0
            elif cases==1:
                arr[index_gate2]=[ 1, gate[1],gate[2]]
                arr.pop(i)
                i=0
            elif cases==2:
                arr[index_gate2]=[ 2, gate[1],gate[2]]
                arr.pop(i)
                i=0
            elif cases==-1:
                i=i+1
        
        elif gate_type==4: #not gate
            i=i+1
        else:
            i=i+1
        r=len(arr)
    arr=get_elements_as_list(arr)
    return arr

def check_swap_cvp(index,gate,arr):
    g_cx=[1,gate[1],gate[2]]
    g_cv=[2,gate[1],gate[2]]

    check_cvp=[i for i, n in enumerate(arr) if n == gate]
    check_cvp=[check_cvp[i] for i, n in enumerate(check_cvp) if n>=index]

    check_cv=[i for i, n in enumerate(arr) if n == g_cv]
    check_cv=[check_cv[i] for i, n in enumerate(check_cv) if n>=index]

    check_cx=[i for i, n in enumerate(arr) if n == g_cx]
    check_cx=[check_cx[i] for i, n in enumerate(check_cx) if n>=index]
    
    if len(check_cvp)>1:
        check_swap_cvp=check_swapping_between_gates(check_cvp[0],check_cvp[1],arr)

    if len(check_cx)>=1:
        check_swap_cx=check_swapping_between_gates(check_cvp[0],check_cx[0],arr)

    if len(check_cv)>=1:
        check_swap_cv=check_swapping_between_gates(check_cvp[0],check_cv[0],arr)
    
    if len(check_cvp)>1 and check_swap_cvp==1:
        cases=1
        check_swap=1
        index_gate2=check_cvp[1]
    elif len(check_cv)>=1 and check_swap_cv==1:
        cases=0
        check_swap=1
        index_gate2=check_cv[0]
    elif len(check_cx)>1 and check_swap_cx==1:
        cases=2
        check_swap=1
        index_gate2=check_cx[0]   
    else:
        cases=-1
        check_swap=0
        index_gate2=-1
    return cases,check_swap,index_gate2

def check_swap_cv(index,gate,arr):
    g_cx=[1,gate[1],gate[2]]
    g_cvp=[3,gate[1],gate[2]]

    check_cv=[i for i, n in enumerate(arr) if n == gate]
    check_cv=[check_cv[i] for i, n in enumerate(check_cv) if n >=index]

    check_cvp=[i for i, n in enumerate(arr) if n == g_cvp]
    check_cvp=[check_cvp[i] for i, n in enumerate(check_cvp) if n>=index]
    
    check_cx=[i for i, n in enumerate(arr) if n == g_cx]
    check_cx=[check_cx[i] for i, n in enumerate(check_cx) if n>=index]

    if len(check_cv)>1:
        check_swap_cv=check_swapping_between_gates(check_cv[0],check_cv[1],arr)
    if len(check_cx)>=1:
        check_swap_cx=check_swapping_between_gates(check_cv[0],check_cx[0],arr)
    if len(check_cvp)>=1:
        check_swap_cvp=check_swapping_between_gates(check_cv[0],check_cvp[0],arr)

    if len(check_cv)>1 and check_swap_cv==1:
        cases=1
        check_swap=1
        index_gate2=check_cv[1]
    elif len(check_cx)>=1 and check_swap_cx==1:
        cases=3
        check_swap=1
        index_gate2=check_cx[0]
    elif len(check_cvp)>=1 and check_swap_cvp==1:
        cases=0
        check_swap=1
        index_gate2=check_cvp[0] 
    else:
       cases=-1
       check_swap=0
       index_gate2=-1
    return cases,check_swap,index_gate2
   
def check_swap_cx(index,gate,arr):
    g_cv=[2,gate[1],gate[2]]
    g_cvp=[3,gate[1],gate[2]]
    
    check_cx=[i for i, n in enumerate(arr) if n == gate]
    check_cx=[check_cx[i] for i, n in enumerate(check_cx) if n >=index]

    check_cv=[i for i, n in enumerate(arr) if n == g_cv]
    check_cv=[check_cv[i] for i, n in enumerate(check_cv) if n>=index]
    
    check_cvp=[i for i, n in enumerate(arr) if n == g_cvp]
    check_cvp=[check_cvp[i] for i, n in enumerate(check_cvp) if n>=index]

    if len(check_cx)>1:
        check_swap_cx=check_swapping_between_gates(check_cx[0],check_cx[1],arr)
    if len(check_cv)>=1 and len(check_cvp)>=1:
        if check_cv[0]<check_cvp[0]:
            check_swap_cv=check_swapping_between_gates(check_cx[0],check_cv[0],arr)
        else:
            check_swap_cvp=check_swapping_between_gates(check_cx[0],check_cvp[0],arr)
    
    elif len(check_cv)>=1:
        check_swap_cv=check_swapping_between_gates(check_cx[0],check_cv[0],arr)
    elif len(check_cvp)>=1:
        check_swap_cvp=check_swapping_between_gates(check_cx[0],check_cvp[0],arr)
    
    if len(check_cx)>1 and check_swap_cx==1:
        cases=0
        check_swap=1
        index_gate2=check_cx[1]
    elif len(check_cv)>=1 and check_swap_cv==1:
        cases=3
        check_swap=1
        index_gate2=check_cv[0]
    elif len(check_cvp)>=1 and check_swap_cvp==1:
        cases=2
        check_swap=1
        index_gate2=check_cvp[0]
    else:
        cases=-1
        check_swap=0
        index_gate2=-1
    return cases,check_swap,index_gate2

def check_swapping_between_gates(start_arr,end_arr,all_gates):
    diff=end_arr-start_arr
    if diff==1:
        check_swap=1
    else:
        gate1=all_gates[start_arr]
        target_gate1=gate1[2]
        control_gate1=gate1[1]
        for i in range(start_arr+1,end_arr):
             gate2=all_gates[i]
             control_gate2=gate2[1]
             target_gate2=gate2[2]
             if target_gate1==control_gate2 or control_gate1==target_gate2:
                check_swap=0
                return
             else:
                 check_swap=1
    return check_swap



def get_elements_as_list(arr):
    for row in arr:
        row[1]=[row[1]]
    return arr

#main algorithm
array_levels_before_decomposition=get_circuit_from_file(f_name)
print("number of wires before=",number_of_wires)
array_levels_after_decomposition_stage1=decomposition_ciruit_into_toffoli_gates(array_levels_before_decomposition)
print("number of wires after=",number_of_wires)
print("array of toffoli gates ",array_levels_after_decomposition_stage1)
array_levels_after_decomposition_stage2=decomposition_into_elementry_gates(array_levels_after_decomposition_stage1)
print("array_levels_after_decomposition_stage2",array_levels_after_decomposition_stage2)
optimized_arr=optimization(array_levels_after_decomposition_stage2)
print(optimized_arr)


number of wires before= 8
number of wires after= 8
array of toffoli gates  [[1, 3, 0, 6], [1, 1, 2, 0], [10, 3, 0, 6], [10, 1, 2, 0], [1, 4, 5, 6], [1, 3, 0, 5], [1, 1, 2, 0], [10, 3, 0, 5], [10, 4, 5, 6], [1, 3, 0, 5], [1, 1, 2, 0], [10, 3, 0, 5], [4, [], 5], [1, 4, 5, 6]]
array_levels_after_decomposition_stage2 [[2, 0, 6], [1, 3, 0], [3, 0, 6], [1, 3, 0], [2, 3, 6], [2, 2, 0], [1, 1, 2], [3, 2, 0], [1, 1, 2], [2, 1, 0], [3, 3, 6], [1, 3, 0], [2, 0, 6], [1, 3, 0], [3, 0, 6], [3, 1, 0], [1, 1, 2], [2, 2, 0], [1, 1, 2], [3, 2, 0], [2, 5, 6], [1, 4, 5], [3, 5, 6], [1, 4, 5], [2, 4, 6], [2, 0, 5], [1, 3, 0], [3, 0, 5], [1, 3, 0], [2, 3, 5], [2, 2, 0], [1, 1, 2], [3, 2, 0], [1, 1, 2], [2, 1, 0], [3, 3, 5], [1, 3, 0], [2, 0, 5], [1, 3, 0], [3, 0, 5], [3, 4, 6], [1, 4, 5], [2, 5, 6], [1, 4, 5], [3, 5, 6], [2, 0, 5], [1, 3, 0], [3, 0, 5], [1, 3, 0], [2, 3, 5], [2, 2, 0], [1, 1, 2], [3, 2, 0], [1, 1, 2], [2, 1, 0], [3, 3, 5], [1, 3, 0], [2, 0, 5], [1, 3, 0], [3, 0, 5], [4, [], 5], [2, 5, 6], [